In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [88]:
import numpy as np
from matplotlib import pylab as plt
import pandas as pd
import imblearn
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [3]:
Beans = pd.read_excel('DryBeanDataSet.xlsx')
Beans.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,Constantness,EquivDiameter,Colour,...,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,ShapeFactor5,ShapeFactor6,Class,Sort order
0,84648,1132.054,439.915710,248.961941,1.767000,0.824453,86779,1,328.294351,brown,...,0.830027,0.746266,0.005197,0.000994,0.556914,3.487008,0.984066,50.809833,CALI,0.134791
1,39704,736.873,271.659919,186.481404,1.456767,0.727175,40106,1,224.839282,black,...,0.918880,0.82765,0.006842,0.001980,0.685004,1.242335,0.997891,148.508874,DERMASON,0.898848
2,35835,687.914,237.704180,192.176759,1.236904,0.588537,36190,1,213.603696,green,...,0.951588,0.898611,0.006633,0.002668,0.807503,2.813489,0.998803,82.107117,SEKER,0.539684
3,223035,1817.278,680.812555,419.994347,1.621004,0.787040,225889,1,532.894907,brown,...,0.848673,0.782734,0.003052,0.000707,0.612672,3.074472,0.993144,147.347735,BOMBAY,0.149130
4,41957,767.143,287.962974,186.166695,1.546802,0.762918,42310,1,231.130508,brown,...,0.895904,0.80264,0.006863,0.001757,0.644230,2.213954,0.996497,22.893826,SIRA,0.447635


In [4]:
Beans.dtypes

Area                 int64
Perimeter          float64
MajorAxisLength    float64
MinorAxisLength    float64
AspectRation       float64
Eccentricity       float64
ConvexArea           int64
Constantness         int64
EquivDiameter      float64
Colour              object
Extent              object
Solidity           float64
roundness          float64
Compactness         object
ShapeFactor1       float64
ShapeFactor2       float64
ShapeFactor3       float64
ShapeFactor4       float64
ShapeFactor5       float64
ShapeFactor6        object
Class               object
Sort order         float64
dtype: object

In [5]:
Beans["Colour"] = Beans["Colour"].astype('category')
Beans["Constantness"] = Beans["Constantness"].astype('category')
Beans["Class"] = Beans["Class"].astype('category')
Beans["Extent"] = pd.to_numeric(Beans["Extent"], errors='coerce')
Beans["Compactness"] = pd.to_numeric(Beans["Compactness"], errors='coerce')
Beans["ShapeFactor6"] = pd.to_numeric(Beans["ShapeFactor6"], errors='coerce')

In [6]:
Beans.dtypes

Area                  int64
Perimeter           float64
MajorAxisLength     float64
MinorAxisLength     float64
AspectRation        float64
Eccentricity        float64
ConvexArea            int64
Constantness       category
EquivDiameter       float64
Colour             category
Extent              float64
Solidity            float64
roundness           float64
Compactness         float64
ShapeFactor1        float64
ShapeFactor2        float64
ShapeFactor3        float64
ShapeFactor4        float64
ShapeFactor5        float64
ShapeFactor6        float64
Class              category
Sort order          float64
dtype: object

In [7]:
Beans.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,Constantness,EquivDiameter,Colour,...,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,ShapeFactor5,ShapeFactor6,Class,Sort order
0,84648,1132.054,439.915710,248.961941,1.767000,0.824453,86779,1,328.294351,brown,...,0.830027,0.746266,0.005197,0.000994,0.556914,3.487008,0.984066,50.809833,CALI,0.134791
1,39704,736.873,271.659919,186.481404,1.456767,0.727175,40106,1,224.839282,black,...,0.918880,0.827650,0.006842,0.001980,0.685004,1.242335,0.997891,148.508874,DERMASON,0.898848
2,35835,687.914,237.704180,192.176759,1.236904,0.588537,36190,1,213.603696,green,...,0.951588,0.898611,0.006633,0.002668,0.807503,2.813489,0.998803,82.107117,SEKER,0.539684
3,223035,1817.278,680.812555,419.994347,1.621004,0.787040,225889,1,532.894907,brown,...,0.848673,0.782734,0.003052,0.000707,0.612672,3.074472,0.993144,147.347735,BOMBAY,0.149130
4,41957,767.143,287.962974,186.166695,1.546802,0.762918,42310,1,231.130508,brown,...,0.895904,0.802640,0.006863,0.001757,0.644230,2.213954,0.996497,22.893826,SIRA,0.447635


In [8]:
Beans.describe()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,ShapeFactor5,ShapeFactor6,Sort order
count,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,1.361100e+04,13605.000000,13611.000000,13611.000000,13593.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13606.000000,13611.000000
mean,53048.284549,855.283459,320.141867,202.270714,1.583242,0.750895,53765.692602,4.762541e+02,0.749747,0.987143,0.873282,0.799886,0.006564,0.001716,0.643590,2.368097,0.995063,89.358603,0.500271
std,29324.095717,214.289696,85.694186,44.970091,0.246678,0.092002,29778.009358,2.583687e+04,0.049085,0.004660,0.059520,0.061684,0.001128,0.000596,0.098996,0.871619,0.004366,51.838555,0.287926
min,20420.000000,524.736000,183.601165,122.512653,1.024868,0.218951,-30.000000,1.614174e-01,0.555315,0.919246,0.489618,0.640577,0.002778,0.000564,0.410339,0.695579,0.947687,0.000466,0.000089
25%,36328.000000,703.523500,253.303633,175.848170,1.432307,0.715928,36714.500000,2.150680e+02,0.718641,0.985670,0.832096,0.762577,0.005900,0.001154,0.581359,1.614151,0.993703,45.258826,0.248187
50%,44652.000000,794.941000,296.883367,192.431733,1.551124,0.764441,45178.000000,2.384380e+02,0.759874,0.988283,0.883157,0.801291,0.006645,0.001694,0.642044,2.368757,0.996386,88.766670,0.503810
75%,61332.000000,977.213000,376.495012,217.031741,1.707109,0.810466,62294.000000,2.794522e+02,0.786852,0.990013,0.916869,0.834270,0.007271,0.002170,0.696006,3.115695,0.997883,134.273148,0.750096
max,254616.000000,1985.370000,738.860153,460.198497,2.430306,0.911423,263261.000000,3.014441e+06,0.866195,0.994677,0.990685,0.987303,0.010451,0.003665,0.974767,3.966119,0.999733,178.985023,0.999985


In [9]:
Beans.std()

C:\Users\andre\AppData\Local\Temp\ipykernel_8848\3045098025.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  Beans.std()


Area               29324.095717
Perimeter            214.289696
MajorAxisLength       85.694186
MinorAxisLength       44.970091
AspectRation           0.246678
Eccentricity           0.092002
ConvexArea         29778.009358
EquivDiameter      25836.865632
Extent                 0.049085
Solidity               0.004660
roundness              0.059520
Compactness            0.061684
ShapeFactor1           0.001128
ShapeFactor2           0.000596
ShapeFactor3           0.098996
ShapeFactor4           0.871619
ShapeFactor5           0.004366
ShapeFactor6          51.838555
Sort order             0.287926
dtype: float64

In [10]:
((Beans.isna().sum())/13611) * 100

Area               0.000000
Perimeter          0.000000
MajorAxisLength    0.000000
MinorAxisLength    0.000000
AspectRation       0.000000
Eccentricity       0.000000
ConvexArea         0.000000
Constantness       0.000000
EquivDiameter      0.000000
Colour             0.000000
Extent             0.044082
Solidity           0.000000
roundness          0.000000
Compactness        0.132246
ShapeFactor1       0.000000
ShapeFactor2       0.000000
ShapeFactor3       0.000000
ShapeFactor4       0.000000
ShapeFactor5       0.000000
ShapeFactor6       0.036735
Class              0.000000
Sort order         0.000000
dtype: float64

In [11]:
(Beans == "?").sum()

Area                0
Perimeter           0
MajorAxisLength     0
MinorAxisLength     0
AspectRation        0
Eccentricity        0
ConvexArea          0
Constantness        0
EquivDiameter       0
Colour              6
Extent              0
Solidity            0
roundness           0
Compactness         0
ShapeFactor1        0
ShapeFactor2        0
ShapeFactor3        0
ShapeFactor4        0
ShapeFactor5        0
ShapeFactor6        0
Class              17
Sort order          0
dtype: int64

In [12]:
Beans.nunique()

Area               12011
Perimeter          13351
MajorAxisLength    13543
MinorAxisLength    13543
AspectRation       13543
Eccentricity       13543
ConvexArea         12066
Constantness           2
EquivDiameter      12012
Colour                 5
Extent             13529
Solidity           13526
roundness          13543
Compactness        13525
ShapeFactor1       13543
ShapeFactor2       13543
ShapeFactor3       13543
ShapeFactor4       13611
ShapeFactor5       13543
ShapeFactor6       13606
Class                  8
Sort order         13611
dtype: int64

In [13]:
cat = Beans[["Constantness", "Colour", "Class"]].copy()
cat

,Constantness,Colour,Class
0,1,brown,CALI
1,1,black,DERMASON
2,1,green,SEKER
3,1,brown,BOMBAY
4,1,brown,SIRA
...,...,...,...
13606,1,black,DERMASON
13607,1,brown,BARBUNYA
13608,0,brown,BARBUNYA
13609,1,white,HOROZ


In [14]:
(((cat == "?").sum())/13611) *100

Constantness    0.000000
Colour          0.044082
Class           0.124899
dtype: float64

In [15]:
# Replace '?' with NaN
cat.replace('?', np.nan, inplace=True)

In [16]:
mode = cat.mode()
print(cat["Class"].value_counts())
print(cat["Colour"].value_counts())
print(cat["Constantness"].value_counts())

DERMASON    3542
SIRA        2634
SEKER       2025
HOROZ       1927
CALI        1628
BARBUNYA    1317
BOMBAY       521
Name: Class, dtype: int64
brown    6115
black    3541
green    2023
white    1926
Name: Colour, dtype: int64
1    12289
0     1322
Name: Constantness, dtype: int64


In [17]:
print((3542/13611)*100)
print((2634/13611)*100)
print((6115/13611)*100)
print((3541/13611)*100)
print((12289/13611)*100)
print((1322/13611)*100)

26.02306957607817
19.3519947101609
44.92689736242745
26.01572257732716
90.2872676511645
9.712732348835502


# Create Dummy variables and replace the missing values with NaN

In [18]:
Beans["Class"].replace('?', np.nan, inplace=True)
Beans["Colour"].replace('?', np.nan, inplace=True)

In [19]:
label = LabelEncoder()

Beans["Class"] = label.fit_transform(Beans["Class"])
Beans["Colour"] = label.fit_transform(Beans["Colour"])
Beans.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,Constantness,EquivDiameter,Colour,...,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,ShapeFactor5,ShapeFactor6,Class,Sort order
0,84648,1132.054,439.915710,248.961941,1.767000,0.824453,86779,1,328.294351,1,...,0.830027,0.746266,0.005197,0.000994,0.556914,3.487008,0.984066,50.809833,2,0.134791
1,39704,736.873,271.659919,186.481404,1.456767,0.727175,40106,1,224.839282,0,...,0.918880,0.827650,0.006842,0.001980,0.685004,1.242335,0.997891,148.508874,3,0.898848
2,35835,687.914,237.704180,192.176759,1.236904,0.588537,36190,1,213.603696,2,...,0.951588,0.898611,0.006633,0.002668,0.807503,2.813489,0.998803,82.107117,5,0.539684
3,223035,1817.278,680.812555,419.994347,1.621004,0.787040,225889,1,532.894907,1,...,0.848673,0.782734,0.003052,0.000707,0.612672,3.074472,0.993144,147.347735,1,0.149130
4,41957,767.143,287.962974,186.166695,1.546802,0.762918,42310,1,231.130508,1,...,0.895904,0.802640,0.006863,0.001757,0.644230,2.213954,0.996497,22.893826,6,0.447635


In [20]:
Beans['Class'] = Beans['Class'].replace(7, np.nan)
Beans['Colour'] = Beans['Colour'].replace(4, np.nan)

In [21]:
print(Beans['Class'].value_counts())
print(Beans['Colour'].value_counts())

3.0    3542
6.0    2634
5.0    2025
4.0    1927
2.0    1628
0.0    1317
1.0     521
Name: Class, dtype: int64
1.0    6115
0.0    3541
2.0    2023
3.0    1926
Name: Colour, dtype: int64


# Handle imbalanced classes

We can drop sort order from the data, as it has unique values for all of the Beans

In [60]:
cleaned_beans = Beans.dropna()
dropped_beans = Beans[Beans.isna().any(axis=1)]
X = cleaned_beans.drop(["Class","Sort order"], axis=1)
y = cleaned_beans["Class"]

In [68]:
smote = SMOTE(random_state=42)
tomek = TomekLinks()


# Initialize SimpleImputer with a strategy (e.g., mean, median, or most_frequent)
imputer = SimpleImputer(strategy='mean')  # or strategy='median', 'most_frequent'

# Fit and transform X to fill in missing values
X_imputed = imputer.fit_transform(X)

# Convert the result back to a DataFrame if needed
X_imputed = pd.DataFrame(X_imputed, columns=X.columns)

# Create a pipeline with SMOTE and Tomek Links
pipeline = Pipeline(steps=[('tomek', tomek), ('smote', smote)])

X_resampled, y_resampled = pipeline.fit_resample(X_imputed,y)

In [73]:
y_resampled.value_counts()

3.0    3204
5.0    3204
1.0    3204
2.0    3204
6.0    3204
4.0    3204
0.0    3204
Name: Class, dtype: int64

In [76]:
X_resampled

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,Constantness,EquivDiameter,Colour,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,ShapeFactor5,ShapeFactor6
0,39704.000000,736.873000,271.659919,186.481404,1.456767,0.727175,40106.000000,1.0,224.839282,0.0,0.757739,0.989977,0.918880,0.827650,0.006842,0.001980,0.685004,1.242335,0.997891,148.508874
1,35835.000000,687.914000,237.704180,192.176759,1.236904,0.588537,36190.000000,1.0,213.603696,2.0,0.790885,0.990191,0.951588,0.898611,0.006633,0.002668,0.807503,2.813489,0.998803,82.107117
2,223035.000000,1817.278000,680.812555,419.994347,1.621004,0.787040,225889.000000,1.0,532.894907,1.0,0.761092,0.987365,0.848673,0.782734,0.003052,0.000707,0.612672,3.074472,0.993144,147.347735
3,34631.000000,670.020000,229.336784,192.401403,1.191970,0.544213,34953.000000,1.0,209.984663,2.0,0.760686,0.990788,0.969392,0.915617,0.006622,0.002871,0.838355,3.268751,0.999294,9.733717
4,32941.000000,695.209000,255.121867,164.681723,1.549181,0.763758,33455.000000,0.0,204.796933,0.0,0.728767,0.984636,0.856477,0.802742,0.007745,0.001984,0.644394,3.751731,0.998283,106.458919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22423,41226.901083,761.650395,281.290708,187.619337,1.499489,0.744764,41745.046327,1.0,229.110712,1.0,0.766166,0.987588,0.893058,0.814581,0.006823,0.001853,0.663609,1.530307,0.994708,138.829040
22424,51101.292028,844.718440,319.376460,204.484089,1.563150,0.766953,51575.411010,1.0,255.076809,1.0,0.766623,0.990807,0.899973,0.798987,0.006250,0.001572,0.638641,2.696354,0.996680,76.751247
22425,50704.691888,852.464898,323.302225,200.422326,1.614094,0.783934,51331.005820,1.0,254.085061,1.0,0.710841,0.987799,0.876873,0.786146,0.006376,0.001503,0.618209,1.482121,0.996632,131.214610
22426,45439.370701,806.764305,311.429507,186.897328,1.666814,0.799525,46038.171952,1.0,240.531087,1.0,0.767134,0.986994,0.877415,0.772481,0.006854,0.001506,0.596833,2.233255,0.994144,99.057740


In [170]:
# # concatenated_df = pd.concat([df1, df2], axis=0)
X_missing = dropped_beans.drop(["Class","Sort order"], axis=1)
y_missing = dropped_beans["Class"]
# X_beans = pd.concat([X_resampled, X_missing], axis=0)
# y_beans = pd.concat([y_resampled, y_missing], axis=0)
# X_standard = (X_beans - X_beans.mean())/X_beans.std()

In [180]:
X_num = X_resampled.drop(["Colour","Constantness"], axis=1)
X_cat = X_resampled[["Colour", "Constantness"]]
X_standard = (X_num - X_num.mean())/X_num.std()

X_beans = pd.concat([X_standard, X_cat], axis=1)
y_beans = y_resampled

In [181]:
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_beans, y_beans, test_size=0.3, random_state=42)

In [197]:
knn = KNeighborsClassifier(n_neighbors=5000)  # n_neighbors is the number of neighbors to use

# Train the classifier
knn.fit(X_train, y_train)
# Make predictions
y_pred = knn.predict(np.asarray(X_test))

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)

C:\Users\andre\anaconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [198]:
accuracy

0.9209392183088126

In [199]:
y_pred = knn.predict(np.asarray(X_missing.dropna()))

y_pred

C:\Users\andre\anaconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])